In [1]:
import pandas as pd
import seaborn as sns
import transformers
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import os, json, gc, re, random
from sklearn.preprocessing import LabelEncoder

In [2]:
import torch, transformers, tokenizers
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from torch.nn import BCEWithLogitsLoss, BCELoss
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score

In [3]:
movie_df = pd.read_csv("kinopoisk-top250g.csv", sep=',')

movies = movie_df

In [4]:
shortlisted_genres = movies["Genre"].value_counts().reset_index(name="count").query("count > 2")["index"].tolist()
movies = movies[movies["Genre"].isin(shortlisted_genres)].reset_index(drop=True)

In [5]:
movies['GenreCorrected'] =movies['Genre'] 
movies['GenreCorrected']=movies['GenreCorrected'].str.strip()
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('драма', 'Драма')
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('Драма ', 'Драма')
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('Детективный', 'Детектив')

In [6]:
movies

,rating,Genre,movie,year,country,rating_ball,Plot,director,screenwriter,actors,url_logo,GenreCorrected
0,0,Драма,Побег из Шоушенка,1994,США,9.111,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Тим Роббинс; Морган Фриман; Боб Гантон; Уильям...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
1,1,Драма,Зеленая миля,1999,США,9.062,Пол Эджкомб — начальник блока смертников в тюр...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Том Хэнкс; Дэвид Морс; Бонни Хант; Майкл Кларк...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
2,2,Драма,Форрест Гамп,1994,США,8.913,От лица главного героя Форреста Гампа; слабоум...,Роберт Земекис,Эрик Рот; Уинстон Грум,Том Хэнкс; Робин Райт; Салли Филд; Гэри Синиз;...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
3,3,Военный,Список Шиндлера,1993,США,8.817,Фильм рассказывает реальную историю загадочног...,Стивен Спилберг,Стивен Зеллиан; Томас Кенилли,Лиам Нисон; Бен Кингсли; Рэйф Файнс; Кэролайн ...,'https://st.kp.yandex.net/images/film_iphone/i...,Военный
4,4,Комедия,1+1,2011,Франция,8.807,Пострадав в результате несчастного случая; бог...,Оливье Накаш; Эрик Толедано,Оливье Накаш; Филипп Поццо ди Борго; Эрик Т...,Франсуа Клюзе; Омар Си; Анн Ле Ни; Одри Флеро;...,'https://st.kp.yandex.net/images/film_iphone/i...,Комедия
...,...,...,...,...,...,...,...,...,...,...,...,...
233,245,Драма,Крамер против Крамера,1979,США,7.984,Тед Крамер; целиком посвятив себя карьере; не ...,Роберт Бентон,Роберт Бентон; Эйвери Кормэн,Дастин Хоффман; Мэрил Стрип; Джейн Александр; ...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
234,246,Приключения,Пираты Карибского моря: На краю Света,2007,США,8.007,Новые приключения Джека Воробья и его друзей У...,Гор Вербински,Тед Эллиот; Терри Россио; Стюарт Битти; ...,Джонни Депп; Джеффри Раш; Орландо Блум; Кира Н...,'https://st.kp.yandex.net/images/film_iphone/i...,Приключения
235,247,Военный,Иди и смотри,1985,СССР,8.077,Флера — шестнадцатилетний мальчишка; откопавши...,Элем Климов,Алесь Адамович; Элем Климов,Алексей Кравченко; Ольга Миронова; Любомирас Л...,'https://st.kp.yandex.net/images/film_iphone/i...,Военный
236,248,Боевик,Профессионал,1981,Франция,8.112,Убить президента африканской республики. Такое...,Жорж Лотнер,Мишель Одиар; Патрик Александер; Жорж Лотне...,Жан-Поль Бельмондо; Жан Дезайи; Сирил Клер; Ма...,'https://st.kp.yandex.net/images/film_iphone/i...,Боевик


In [8]:
movies['genre_new'] = [x.replace('|',',').split(',') for x in movies['GenreCorrected'].values]
movies['genre_new'] = movies['genre_new'].apply(lambda genres: [x.strip() for x in genres])
movies['genre_new'] = movies['genre_new'].apply(lambda genres: list(filter(lambda x: len(x) != 0, genres)))
movies['genre_new'].iloc[30]

['Детский']

In [10]:
movies = movies[["Plot", "GenreCorrected", "genre_new"]]

In [11]:
multilabel_binarizer = MultiLabelBinarizer()
genre_encoded = multilabel_binarizer.fit_transform(movies['genre_new'])

values = movies[["Plot", "GenreCorrected", "genre_new"]].values
values = [[values[i][0], values[i][1], values[i][2], genre_encoded[i]] for i in range(len(values)) ]

movies = pd.DataFrame(values, columns=['text', 'GenreCorrected', "genre_new", "labels"])

In [12]:
len(movies), len(multilabel_binarizer.classes_)

(238, 14)

In [13]:
%%time

from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs

args = MultiLabelClassificationArgs()
args.reprocess_input_data = True
args.overwrite_output_dir = True
args.save_model_every_epoch = False
args.save_model_every_epoch = False
args.max_seq_length = 512
args.train_batch_size = 4
args.num_train_epochs = 4
args.threshold = 0.26


# Create a ClassificationModel
model = MultiLabelClassificationModel('bert', 'bert-base-cased', num_labels=len(multilabel_binarizer.classes_), args=args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not in

Wall time: 1min 55s


In [14]:
train_df, eval_df = train_test_split(movies, test_size=0.1, random_state=42)
train_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)
print("train: ", len(train_df))
print("evaluation: ", len(eval_df))

train:  214
evaluation:  24


In [15]:
%%time
model.train_model(train_df[["text", "labels"]])

  0%|          | 0/214 [00:00<?, ?it/s]

E:\Program Files\Anaconda\lib\site-packages\simpletransformers\classification\classification_utils.py:267: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  labels = torch.tensor(labels, dtype=torch.long)


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/54 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/54 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/54 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/54 [00:00<?, ?it/s]

Wall time: 3min 38s


(216, 0.28517535439244024)

In [16]:
result, model_outputs, wrong_output = model.eval_model(eval_df[["text", "labels"]])

  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
# https://scikit-learn.org/stable/modules/model_evaluation.html#label-ranking-average-precision
print(result)

{'LRAP': 0.34811403873903873, 'eval_loss': 0.255087877313296}


In [18]:
corr = 0
for i in range(len(eval_df)):
    if ((model_outputs[i] > 0.3).astype(int) == np.array(eval_df['labels'][i])).all():
        corr+=1
print("acc: {}".format(corr / len(eval_df)))

acc: 0.0


In [20]:
for _ in range(10):
    random_idx = random.randint(0, len(eval_df)-1)
    text = eval_df.iloc[random_idx]['text']
    true_genre = eval_df.iloc[random_idx]['GenreCorrected']
    predicted_genre_encoded, _ = model.predict([text])
    predicted_genre = multilabel_binarizer.inverse_transform(np.array(predicted_genre_encoded))

    print('\nTrue Genre: {}\n'.format(true_genre))
    print('Predicted Genre: {}\n'.format(predicted_genre))
    print('Plot: {}\n'.format(text))
    print("-------------------------------------------")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


True Genre: Драма

Predicted Genre: [()]

Plot: Это правдивая и печальная история жизни реально существовавшего в 19 веке человека; Джона Меррика; страдавшего редчайшей болезнью; вызывавшей чудовищные деформации тела.Молодой хирург Тривес; увидев это несчастное и забитое существо в балагане; выкупает его у хозяина из чисто научного интереса; но с удивлением обнаруживает в нем человека интеллектуально развитого и духовного. Профессиональный интерес сменяется состраданием; феномен становится моден в высшем свете; сама королева благословляет оставить его в больнице навсегда…

-------------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]


True Genre: Детектив

Predicted Genre: [()]

Plot: Вена. Начало 20 века. В городе появляется загадочный человек; называвший себя иллюзионистом Эйзенхаймом. Он показывает публике невиданные фокусы; которые кажутся не иначе как волшебством. Слава о необыкновенном чародее доходит до кронпринца Леопольда; который почтил своим присутствием одно из представлений Эйзенхайма. Леопольда сопровождает его будущая невеста Софи. Её появление в зале дает толчок для целой вереницы необъяснимых событий; корни которых уходят в прошлое…

-------------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


True Genre: Детектив

Predicted Genre: [()]

Plot: Роберт и Альфред — фокусники-иллюзионисты; которые на рубеже XIX и XX веков соперничали друг с другом в Лондоне. С годами их дружеская конкуренция на профессиональной почве перерастает в настоящую войну.Они готовы на все; чтобы выведать друг у друга секреты фантастических трюков и сорвать их исполнение. Непримиримая вражда; вспыхнувшая между ними; начинает угрожать жизни окружающих их людей…

-------------------------------------------
